In [46]:
import math  # Just ignore this :-)
import numpy as np

# ML E2021 - Week 10 - Practical Exercises

In the exercise below, you will see an example of how a hidden Markov model (HMM)
can be represented, and you will implement and experiment with the computation of the joint probability and various decodings as explained in the lectures in week 9.

# 1 - Representing an HMM

We can represent a HMM as a triple consisting of three matrices: a $K \times 1$ matrix with the initial state probabilities, a $K \times K$ matrix with the transition probabilities and a $K \times |\Sigma|$ matrix with the emission probabilities. In Python we can write the matrices like this:

In [47]:
init_probs_7_state = [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00]

trans_probs_7_state = [
    [0.00, 0.00, 0.90, 0.10, 0.00, 0.00, 0.00],
    [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.05, 0.90, 0.05, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
    [0.00, 0.00, 0.00, 0.10, 0.90, 0.00, 0.00],
]

emission_probs_7_state = [
    #   A     C     G     T
    [0.30, 0.25, 0.25, 0.20],
    [0.20, 0.35, 0.15, 0.30],
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
    [0.30, 0.20, 0.30, 0.20],
    [0.15, 0.30, 0.20, 0.35],
]

How do we use these matrices? Remember that we are given some sequence of observations, e.g. like this:

In [48]:
obs_example = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'

To make a lookup in our three matrices, it is convenient to translate each symbol in the string to an index.

In [49]:
def translate_observations_to_indices(obs):
    mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    return [mapping[symbol.lower()] for symbol in obs]

Let's try to translate the example above using this function:

In [50]:
obs_example_trans = translate_observations_to_indices(obs_example)

In [51]:
obs_example_trans

[2,
 3,
 3,
 3,
 1,
 1,
 1,
 0,
 2,
 3,
 2,
 3,
 0,
 3,
 0,
 3,
 1,
 2,
 0,
 2,
 2,
 2,
 0,
 3,
 0,
 1,
 3,
 0,
 1,
 2,
 3,
 2,
 1,
 0,
 3,
 0,
 2,
 3,
 0,
 0,
 1,
 0,
 3,
 1,
 2,
 2,
 1,
 1,
 0,
 0]

Use the function below to translate the indices back to observations:

In [52]:
def translate_indices_to_observations(indices):
    mapping = ['a', 'c', 'g', 't']
    return ''.join(mapping[idx] for idx in indices)

In [53]:
translate_indices_to_observations(translate_observations_to_indices(obs_example))

'gtttcccagtgtatatcgagggatactacgtgcatagtaacatcggccaa'

Now each symbol has been transformed (predictably) into a number which makes it much easier to make lookups in our matrices. We'll do the same thing for a list of states (a path):

In [54]:
def translate_path_to_indices(path):
    return list(map(lambda x: int(x), path))

def translate_indices_to_path(indices):
    return ''.join([str(i) for i in indices])

Given a path through an HMM, we can now translate it to a list of indices:

In [55]:
path_example = '33333333333321021021021021021021021021021021021021'

translate_path_to_indices(path_example)

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 2,
 1]

Finally, we can collect the three matrices in a class to make it easier to work with our HMM.

In [56]:
class hmm:
    def __init__(self, init_probs, trans_probs, emission_probs):
        self.init_probs = init_probs
        self.trans_probs = trans_probs
        self.emission_probs = emission_probs

# Collect the matrices in a class.
hmm_7_state = hmm(init_probs_7_state, trans_probs_7_state, emission_probs_7_state)

# We can now reach the different matrices by their names. E.g.:
hmm_7_state.trans_probs

[[0.0, 0.0, 0.9, 0.1, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.05, 0.9, 0.05, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.1, 0.9, 0.0, 0.0]]

For testing, here's another model (which we will refer to as the 3-state model).

In [57]:
init_probs_3_state = [0.10, 0.80, 0.10]

trans_probs_3_state = [
    [0.90, 0.10, 0.00],
    [0.05, 0.90, 0.05],
    [0.00, 0.10, 0.90],
]

emission_probs_3_state = [
    #   A     C     G     T
    [0.40, 0.15, 0.20, 0.25],
    [0.25, 0.25, 0.25, 0.25],
    [0.20, 0.40, 0.30, 0.10],
]

hmm_3_state = hmm(init_probs_3_state, trans_probs_3_state, emission_probs_3_state)

# 2 - Validating an HMM (and handling floats)

Before using the model we'll write a function to validate that the model is valid. That is, the matrices should have the right dimensions and the following things should be true:

1. The initial probabilities must sum to 1.
2. Each row in the matrix of transition probabilities must sum to 1.
3. Each row in the matrix of emission probabilities must sum to 1.
4. All numbers should be between 0 and 1, inclusive.

Write a function `validate_hmm` that given a model returns True if the model is valid, and False otherwise:

In [58]:
def validate_hmm(model):
    pi =np.array( model.init_probs)
    A = np.array(model.trans_probs)
    fi =np.array(model.emission_probs)
    for i in range(A.shape[0]):
        rowSum = 0
        for j in range(A.shape[1]):
            rowSum += A[i,j]
        if not(np.allclose(rowSum, 1)):
            return False
    sum = 0
    for i in range(pi.shape[0]):
        sum += pi[i]
    if not(np.allclose(sum, 1)):
        print("Oh no")
    for i in range(fi.shape[0]):
        rowSum = 0
        for j in range(fi.shape[1]):
            rowSum += fi[i,j]
        if not(np.allclose(rowSum, 1)):
            return False
    return True

We can now use this function to check whether the example model is a valid model.

In [59]:
validate_hmm(hmm_7_state)

True

You might run into problems related to summing floating point numbers because summing floating point numbers does not (always) give the expected result as illustrated by the following examples. How do you suggest to deal with this?

In [60]:
0.15 + 0.30 + 0.20 + 0.35

0.9999999999999999

The order of the terms matter.

In [61]:
0.20 + 0.35 + 0.15 + 0.30

1.0

Because it changes the prefix sums

In [62]:
0.15 + 0.30

0.44999999999999996

In [63]:
0.20 + 0.35 + 0.15

0.7000000000000001

In [64]:
0.15 + 0.30

0.44999999999999996

On should never compare floating point numbers. They represent only an 'approximation'. Read more about the 'problems' in 'What Every Computer Scientist Should Know About Floating-Point Arithmetic' at:

http://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html

# 3 - Computing the Joint Probability

Recall that the joint probability $p({\bf X}, {\bf Z}) = p({\bf x}_1, \ldots, {\bf x}_N, {\bf z}_1, \ldots, {\bf z}_N)$ of a hidden Markov model (HMM) can be compute as

$$
p({\bf x}_1, \ldots, {\bf x}_N, {\bf z}_1, \ldots, {\bf z}_N) = p({\bf z}_1) 
\left[ \prod_{n=2}^N p({\bf z}_n \mid {\bf z}_{n-1}) \right]
\prod_{n=1}^N p({\bf x}_n \mid {\bf z}_n)
$$

## Implementing without log-transformation

Write a function `joint_prob` given a model (e.g. in the representation above) and sequence of observables, ${\bf X}$, and a sequence of hidden states, ${\bf Z}$, computes the joint probability cf. the above formula.

In [65]:
def joint_prob(model, x, z):
    observeList = translate_observations_to_indices(x)
    pathList = translate_path_to_indices(z)
    pi = model.init_probs
    A = model.trans_probs
    fi = model.emission_probs
    proba = 1
    for i in range(len(observeList)):
        if i == 0:
            proba = proba* pi[pathList[i]]
            proba = proba* fi[pathList[i]][observeList[i]]
        else:
            proba *= A[pathList[i-1]][pathList[i]]
            proba *= fi[pathList[i]][observeList[i]]
    return proba


def joint_prob1(model, x, z):
    x = translate_observations_to_indices(x)
    z = translate_path_to_indices(z)
    initial_probs = model.init_probs
    trans_probs = model.trans_probs
    emission_probs = model.emission_probs
    initial_z = initial_probs[z[0]]
    transprobs = [trans_probs[z[i-1]][z[i]] for i in range(1, len(z))]
    emissionprobs = [emission_probs[z[i]][x[i]] for i in range(len(z))]
    return initial_z * math.prod(transprobs) * math.prod(emissionprobs)     
        

Now compute the joint probability of the ${\bf X}$ (`x_short`) and ${\bf Z}$ (`z_short`) below using the 7-state (`hmm_7_state`) model introduced above. (*Remember to translate them first using the appropriate functions introduces above!*)

In [66]:
x_short = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
z_short = '33333333333321021021021021021021021021021021021021'

# Your code here ...
print(joint_prob(hmm_7_state, x_short, z_short))
print(joint_prob1(hmm_7_state, x_short, z_short))

1.9114255184318858e-31
1.9114255184318878e-31


## Implementing with log-transformation (i.e. in "log-space")

Now implement the joint probability function using log-transformation as explained in the lecture. We've given you a log-function that handles $\log(0)$.

In [67]:
def log(x):
    if x == 0:
        return float('-inf')
    return math.log(x)


def joint_prob_log(model, x, z):

    model.init_probs = np.vectorize(log)(model.init_probs)
    A = model.trans_probs
    fi = model.emission_probs
    vfunc = np.vectorize(log)
    newPi = vfunc(fi)
    print(newPi)

    return model

Confirm that the log-transformed function is correct by comparing the output of `joint_prob_log` to the output of `joint_prob`.

In [68]:
# Your code here ...
x_short = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
z_short = '33333333333321021021021021021021021021021021021021'

# Your code here ...
#joint_prob(hmm_7_state, x_short, z_short)
#joint_prob_log(hmm_7_state, x_short, z_short)

## Comparison of Implementations

Now that you have two ways to compute the joint probability given a model, a sequence of observations, and a sequence of hidden states, try to make an experiment to figure out how long a sequence can be before it becomes essential to use the log-transformed version. For this experiment we'll use two longer sequences.

In [69]:
x_long = 'TGAGTATCACTTAGGTCTATGTCTAGTCGTCTTTCGTAATGTTTGGTCTTGTCACCAGTTATCCTATGGCGCTCCGAGTCTGGTTCTCGAAATAAGCATCCCCGCCCAAGTCATGCACCCGTTTGTGTTCTTCGCCGACTTGAGCGACTTAATGAGGATGCCACTCGTCACCATCTTGAACATGCCACCAACGAGGTTGCCGCCGTCCATTATAACTACAACCTAGACAATTTTCGCTTTAGGTCCATTCACTAGGCCGAAATCCGCTGGAGTAAGCACAAAGCTCGTATAGGCAAAACCGACTCCATGAGTCTGCCTCCCGACCATTCCCATCAAAATACGCTATCAATACTAAAAAAATGACGGTTCAGCCTCACCCGGATGCTCGAGACAGCACACGGACATGATAGCGAACGTGACCAGTGTAGTGGCCCAGGGGAACCGCCGCGCCATTTTGTTCATGGCCCCGCTGCCGAATATTTCGATCCCAGCTAGAGTAATGACCTGTAGCTTAAACCCACTTTTGGCCCAAACTAGAGCAACAATCGGAATGGCTGAAGTGAATGCCGGCATGCCCTCAGCTCTAAGCGCCTCGATCGCAGTAATGACCGTCTTAACATTAGCTCTCAACGCTATGCAGTGGCTTTGGTGTCGCTTACTACCAGTTCCGAACGTCTCGGGGGTCTTGATGCAGCGCACCACGATGCCAAGCCACGCTGAATCGGGCAGCCAGCAGGATCGTTACAGTCGAGCCCACGGCAATGCGAGCCGTCACGTTGCCGAATATGCACTGCGGGACTACGGACGCAGGGCCGCCAACCATCTGGTTGACGATAGCCAAACACGGTCCAGAGGTGCCCCATCTCGGTTATTTGGATCGTAATTTTTGTGAAGAACACTGCAAACGCAAGTGGCTTTCCAGACTTTACGACTATGTGCCATCATTTAAGGCTACGACCCGGCTTTTAAGACCCCCACCACTAAATAGAGGTACATCTGA'
z_long = '3333321021021021021021021021021021021021021021021021021021021021021021033333333334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021033334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456332102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102103210210210210210210210210210210210210210210210210210210210210210'

Now compute the joint probability with `joint_prob` the 7-state (hmm_7_state) model introduced above, and see when it breaks (i.e. when it wrongfully becomes 0). Does this make sense? Here's some code to get you started.

In [70]:
for i in range(0, len(x_long), 100):
    x = x_long[:i]
    z = z_long[:i]
    
    x_trans = translate_observations_to_indices(x)
    z_trans = translate_path_to_indices(z)
    
    # Make your experiment here...

In the cell below you should state for which $i$ computing the joint probability (for the two models considered) using `joint_prob` wrongfully becomes 0.

**Your answer here:**

For the 7-state model, `joint_prob` becomes 0 for **i = ?**.

# 4 - Viterbi Decoding

Below you will implement and experiment with the Viterbi algorithm. The implementation has been split into three parts:

1. Fill out the $\omega$ table using the recursion presented at the lecture.
2. Find the state with the highest probability after observing the entire sequence of observations.
3. Backtrack from the state found in the previous step to obtain the optimal path.

We'll be working with the 7-state model (`hmm_7_state`) and the helper function for translating between observations, hidden states, and indicies, as introduced above.

Additionally, you're given the function below that constructs a table of a specific size filled with zeros.

In [71]:
def make_table(m, n):
    """Make a table with `m` rows and `n` columns filled with zeros."""
    return [[0] * n for _ in range(m)]

You'll be testing your code with the same two sequences as above, i.e:

In [72]:
x_short = 'GTTTCCCAGTGTATATCGAGGGATACTACGTGCATAGTAACATCGGCCAA'
z_short = '33333333333321021021021021021021021021021021021021'

In [73]:
x_long = 'TGAGTATCACTTAGGTCTATGTCTAGTCGTCTTTCGTAATGTTTGGTCTTGTCACCAGTTATCCTATGGCGCTCCGAGTCTGGTTCTCGAAATAAGCATCCCCGCCCAAGTCATGCACCCGTTTGTGTTCTTCGCCGACTTGAGCGACTTAATGAGGATGCCACTCGTCACCATCTTGAACATGCCACCAACGAGGTTGCCGCCGTCCATTATAACTACAACCTAGACAATTTTCGCTTTAGGTCCATTCACTAGGCCGAAATCCGCTGGAGTAAGCACAAAGCTCGTATAGGCAAAACCGACTCCATGAGTCTGCCTCCCGACCATTCCCATCAAAATACGCTATCAATACTAAAAAAATGACGGTTCAGCCTCACCCGGATGCTCGAGACAGCACACGGACATGATAGCGAACGTGACCAGTGTAGTGGCCCAGGGGAACCGCCGCGCCATTTTGTTCATGGCCCCGCTGCCGAATATTTCGATCCCAGCTAGAGTAATGACCTGTAGCTTAAACCCACTTTTGGCCCAAACTAGAGCAACAATCGGAATGGCTGAAGTGAATGCCGGCATGCCCTCAGCTCTAAGCGCCTCGATCGCAGTAATGACCGTCTTAACATTAGCTCTCAACGCTATGCAGTGGCTTTGGTGTCGCTTACTACCAGTTCCGAACGTCTCGGGGGTCTTGATGCAGCGCACCACGATGCCAAGCCACGCTGAATCGGGCAGCCAGCAGGATCGTTACAGTCGAGCCCACGGCAATGCGAGCCGTCACGTTGCCGAATATGCACTGCGGGACTACGGACGCAGGGCCGCCAACCATCTGGTTGACGATAGCCAAACACGGTCCAGAGGTGCCCCATCTCGGTTATTTGGATCGTAATTTTTGTGAAGAACACTGCAAACGCAAGTGGCTTTCCAGACTTTACGACTATGTGCCATCATTTAAGGCTACGACCCGGCTTTTAAGACCCCCACCACTAAATAGAGGTACATCTGA'
z_long = '3333321021021021021021021021021021021021021021021021021021021021021021033333333334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210210321021021021021021021021033334564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564564563333333456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456456332102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102102103210210210210210210210210210210210210210210210210210210210210210'

Remember to translate these sequences to indices before using them with your algorithms.

## Implementing without log-transformation

First, we will implement the algorithm without log-transformation. This will cause issues with numerical stability (like above when computing the joint probability), so we will use the log-transformation trick to fix this in the next section.

### Computation of the $\omega$ table

In [74]:
def compute_w(model, X):
    x = translate_observations_to_indices(X)
    K = len(model.init_probs)
    n = len(x)
    
    
    w = np.zeros((K, n))
    
    #print("x: ", x)
    # Base case: fill out w[i][0] for i = 0..k-1
    # ...
    for i in range(K): 
        w[i,0] = model.init_probs[i] * model.emission_probs[i][x[0]] #Prob of state z_i * prob x[0] given z_i
    #print("w after base case: ", w[:,0])
    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    # ...
    for col in range(1,n): #For each column in omega
        for row in range(K): #For each row (each z-state) in omega
            max_so_far = 0
            #max = np.max(w[:][col-1] * model.trans_probs[:][row])
            for z_i in range(K): #For each row in previous column
                prev_prob = w[z_i,col-1] * model.trans_probs[z_i][row]
                if prev_prob > max_so_far: 
                    max_so_far = prev_prob
            w[row,col] = model.emission_probs[row][x[col]] * max_so_far
    return w #TADAAA XD

### Finding the joint probability of an optimal path

Now, write a function that given the $\omega$-table, returns the probability of an optimal path through the HMM. As explained in the lecture, this corresponds to finding the highest probability in the last column of the table.

In [75]:
def opt_path_prob(w):
    n = w.shape[1]
    probForLastZ = np.max(w[:, n-1]) #... I gave you my heart, but the very next day you gave it awayyyyyy.... Thiiiiiis year...
    return probForLastZ
    

Now test your implementation in the box below:

In [76]:
w = compute_w(hmm_7_state, x_short)
opt_path_prob(w)
w.T

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.50000000e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 3.12500000e-03, 5.62500000e-02,
        1.25000000e-03, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.37500000e-04, 7.03125000e-04, 1.26562500e-02,
        2.81250000e-04, 2.50000000e-04, 0.00000000e+00],
       [1.87500000e-04, 2.10937500e-04, 1.58203125e-04, 2.84765625e-03,
        6.32812500e-05, 5.62500000e-05, 8.75000000e-05],
       [5.27343750e-05, 5.53710938e-05, 2.53125000e-05, 6.40722656e-04,
        5.69531250e-05, 1.26562500e-05, 1.68750000e-05],
       [1.38427734e-05, 8.85937500e-06, 7.11914063e-06, 1.44162598e-04,
        1.28144531e-05, 1.13906250e-05, 3.79687500e-06],
       [2.21484375e-06, 2.49169922e-06, 1.86877441e-06, 3.24365845e-05,
        2.88325195e-06, 2.56289063e-06, 3.41718750e-06],
       [7.47509766e-07, 3.73754883e-07, 7.97343750e-07, 7.29823151e-06,
        6.15093750e-07, 8

Now do the same for `x_long`. What happens?

In [77]:
w = compute_w(hmm_7_state, x_long)
opt_path_prob(w)

0.0

### Obtaining an optimal path through backtracking

Implement backtracking to find a most probable path of hidden states given the $\omega$-table.

In [78]:
def backtrack(model, X, w):
    pathList = []
    n = w.shape[1]
    x = translate_observations_to_indices(X)

    #n-1
    lastZ = np.argmax(w[:, n-1])
    lastZProb = np.max(w[:, n-1])
    pathList.insert(0,lastZ)
    print(pathList)

    #n-2 to 1
    prev = lastZ
    prevProb = lastZProb
    for col in range(n-2, -1, -1):
        columnw = w[:, col]
        emissionProb = model.emission_probs[prev][x[col+1]]
        for index, val in enumerate(columnw): 
            currentProb = emissionProb * (val * model.trans_probs[index][prev])
            if prevProb == currentProb:
                prevProb = val
                prev = index
                pathList.insert(0,prev)
                break

    realPath = translate_indices_to_path(pathList)        
    return realPath


In [79]:
w = compute_w(hmm_7_state, x_short)
z_viterbi = backtrack(hmm_7_state, x_short, w)
print(z_viterbi)
print(z_short)

[1]
33333333333321021021021021021021021021021021021021
33333333333321021021021021021021021021021021021021


Now do the same for `x_long`. What happens?

## Implementing with log-transformation

Now implement the Viterbi algorithm with log-transformation. The steps are the same as above.

### Computation of the (log-transformed) $\omega$ table

In [80]:
def log(x):
        if x == 0:
            return float('-inf')
        #print("dis x:",x)
        return np.log(x)

def nlog(x):
    return np.vectorize(log)(x)

def compute_w_log(model, X): 
    init_probs = nlog(model.init_probs)
    emission_probs = nlog(model.emission_probs)
    trans_probs = nlog(model.trans_probs)

    x = translate_observations_to_indices(X)
    K = len(model.init_probs)
    n = len(x)

    w = np.zeros((K, n))
    # BASE CASE: fill out w[i][0] for i = 0..k-1
    for i in range(K): 
        w[i,0] = init_probs[i] + emission_probs[i][x[0]] #Prob of state z_i * prob x[0] given z_i

    # Inductive case: fill out w[i][j] for i = 0..k, j = 0..n-1
    for col in range(1,n): #For each column in omega
        for row in range(K): #For each row (each z-state) in omega
            max_so_far = float('-inf')
            #max = np.max(w[:][col-1] * model.trans_probs[:][row])
            for z_i in range(K): #For each row in previous column
                prev_prob = w[z_i,col-1] + trans_probs[z_i][row]
                if prev_prob > max_so_far: 
                    max_so_far = prev_prob
            w[row,col] = emission_probs[row][x[col]] + max_so_far
    return w 

### Finding the (log-transformed) joint probability of an optimal path

In [81]:
def opt_path_prob_log(w):
    n = w.shape[1]
    probForLastZ = np.max(w[:, n-1]) #... I gave you my heart, but the very next day you gave it awayyyyyy.... Thiiiiiis year...
    return probForLastZ

In [82]:
w = compute_w_log(hmm_7_state, x_short)
opt_path_prob_log(w)

-70.73228857440488

Now do the same for `x_long`. What happens?

In [83]:
# Your code here ...
w = compute_w_log(hmm_7_state, x_long)
opt_path_prob_log(w)

-1406.7209253880144

### Obtaining an optimal path through backtracking

In [84]:
def backtrack_log(model, X, w):
    #init_probs = nlog(model.init_probs)
    emission_probs = nlog(model.emission_probs)
    trans_probs = nlog(model.trans_probs)

    pathList = []
    n = w.shape[1]
    x = translate_observations_to_indices(X)

    #n-1
    lastZ = np.argmax(w[:, n-1])
    lastZProb = np.max(w[:, n-1])
    pathList.insert(0,lastZ)
    print(pathList)

    #n-2 to 1
    prev = lastZ
    prevProb = lastZProb
    for col in range(n-2, -1, -1):
        columnw = w[:, col]
        emissionProb = emission_probs[prev][x[col+1]]
        for index, val in enumerate(columnw): 
            currentProb = emissionProb + (val + trans_probs[index][prev])
            if prevProb == currentProb:
                prevProb = val
                prev = index
                pathList.insert(0,prev)
                break

    realPath = translate_indices_to_path(pathList)        
    return realPath

In [85]:
w = compute_w_log(hmm_7_state, x_short)
z_viterbi_log = backtrack_log(hmm_7_state, x_short, w)
w = compute_w(hmm_7_state, x_short)
z_viterbi = backtrack(hmm_7_state, x_short, w)

z_viterbi_log, z_viterbi

[1]
[1]


('33333333333321021021021021021021021021021021021021',
 '33333333333321021021021021021021021021021021021021')

Now do the same for `x_long`. What happens?

In [86]:
# Your code here ...

### Does it work?

Think about how to verify that your implementations of Viterbi (i.e. `compute_w`, `opt_path_prob`, `backtrack`, and there log-transformed variants `compute_w_log`, `opt_path_prob_log`, `backtrack_log`) are correct.

One thing that should hold is that the probability of a most likely path as computed by `opt_path_prob` (or `opt_path_prob_log`) for a given sequence of observables (e.g. `x_short` or `x_long`) should be equal to the joint probability of a corersponding most probable path as found by `backtrack` (or `backtrack_log`) and the given sequence of observables. Why?

Make an experiment that validates that this is the case for your implementations of Viterbi and `x_short` and `x_long`.

In [87]:
# Check that opt_path_prob is equal to joint_prob(hmm_7_state, x_short, z_viterbi)

# Your code here ...

# Check that opt_path_prob_log is equal to joint_prob_log(hmm_7_state, x_short, z_viterbi_log)

# Your code here ...

# Do the above checks for x_long ...

# Your code here ...

Do your implementations pass the above checks?

### Does log-transformation matter?

Make an experiment that investigates how long the input string can be before `backtrack` and `backtrack_log` start to disagree on a most likely path and its probability.

In [88]:
# Your code here ...

**Your answer here:**

For the 7-state model, `backtrack` and `backtrack_log` start to disagree on a most likely path and its probability for **i = ?** .

# 5 - Posterior Decoding

If you have time, try to implement posterior decoding (with scaling, if possible) as explained in the lecture

In [89]:
def forward(model, x):
    pass

def backward(model, x):
    pass

def posterior_decoding(model, x):
    pass

Compare the Viterbi and posterior decodings for the `x_short` and `x_long` using the 7-state model (`hmm_7_state`).

In [90]:
# Your code here ...


a = nlog([0,0])
a[0]*a[1]

inf